<a href="https://colab.research.google.com/github/05kashyap/05kashyap/blob/main/fns2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install simplet5

In [2]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Data preprocessing

In [13]:
df = pd.read_csv("/content/drive/MyDrive/fns_2020.csv")

In [14]:
df.head()

,Gold Summaries,Annual Reports
0,chairman‘s statement\nit has always been your...,mining corporation plc annual report account...
1,pan african resources plc integrated annual r...,the african focused precious metals producer\...
2,note the operating profit operating margin ad...,wwwtribalgroupcouk\ntribal group plc\n8791 ne...
3,significant progress in the year under review...,msb annual report 2005\nintegrated recruitmen...
4,anite annual report and accounts 2012\n08\nou...,anite plc\nannual report and accounts 2012 6...


In [15]:
!pip install nltk
!pip install spacy
!python -m spacy download en_core_web_sm


2023-10-29 04:39:58.940901: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-29 04:39:58.940955: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-29 04:39:58.940988: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-10-29 04:39:58.960987: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-29 04:40:01.042233: W tensorflow/compiler/

# Basic preprocessing, testing on simple t5

In [16]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
import string

In [17]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [18]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [19]:
def preprocess_text(text):
    # Tokenization
    tokens = word_tokenize(text)

    # Remove punctuation and convert to lowercase
    tokens = [word.lower() for word in tokens if word.isalpha()]

    # Remove stop words
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]

    # Stemming
    ps = PorterStemmer()
    stemmed_words = [ps.stem(word) for word in tokens]

    # Join the tokens back into a single string
    processed_text = ' '.join(stemmed_words)

    return processed_text

In [20]:
df.shape

(2576, 2)

In [21]:
df = df.iloc[:100,:]

In [22]:
df['Annual Reports'] = df['Annual Reports'].apply(preprocess_text)

In [23]:
df.head()

,Gold Summaries,Annual Reports
0,chairman‘s statement\nit has always been your...,mine corpor plc annual report account toledo m...
1,pan african resources plc integrated annual r...,african focus preciou metal produc pan african...
2,note the operating profit operating margin ad...,wwwtribalgroupcouk tribal group plc newman str...
3,significant progress in the year under review...,msb annual report integr recruit servic financ...
4,anite annual report and accounts 2012\n08\nou...,anit plc annual report account consolid cash ﬂ...


In [24]:
df['Gold Summaries'] = df['Gold Summaries'].apply(preprocess_text)

In [25]:
df.head()

,Gold Summaries,Annual Reports
0,chairman statement alway compani intent maximi...,mine corpor plc annual report account toledo m...
1,pan african resourc plc integr annual report p...,african focus preciou metal produc pan african...
2,note oper profit oper margin adjust profit tax...,wwwtribalgroupcouk tribal group plc newman str...
3,signific progress year review made line strate...,msb annual report integr recruit servic financ...
4,anit annual report account strategi progress b...,anit plc annual report account consolid cash ﬂ...


In [26]:
from sklearn.model_selection import train_test_split

In [27]:
df = df.rename(columns={"Gold Summaries":"target_text", "Annual Reports":"source_text"})
df = df[['source_text', 'target_text']]

# T5 model expects a task related prefix: since it is a summarization task, we will add a prefix "summarize: "
df['source_text'] = "summarize: " + df['source_text']
df

,source_text,target_text
0,summarize: mine corpor plc annual report accou...,chairman statement alway compani intent maximi...
1,summarize: african focus preciou metal produc ...,pan african resourc plc integr annual report p...
2,summarize: wwwtribalgroupcouk tribal group plc...,note oper profit oper margin adjust profit tax...
3,summarize: msb annual report integr recruit se...,signific progress year review made line strate...
4,summarize: anit plc annual report account cons...,anit annual report account strategi progress b...
...,...,...
95,summarize: n brown group plc annual report acc...,n brown group plc annual report account chief ...
96,summarize: annual report pleas visit wwwmorsec...,mors plc annual report mors plc annual report ...
97,summarize: senior plc annual report account se...,senior plc annual report account chairman stat...
98,summarize: annual report account portfolio ana...,wwwstatprocom page growth chairman statement p...


In [28]:
train_df, test_df = train_test_split(df, test_size=0.2)
train_df.shape, test_df.shape

((80, 2), (20, 2))

In [29]:
from simplet5 import SimpleT5

model = SimpleT5()
model.from_pretrained(model_type="t5", model_name="t5-base")
model.train(train_df=train_df[:5000],
            eval_df=test_df[:100],
            source_max_token_len=128,
            target_max_token_len=50,
            batch_size=8, max_epochs=3, use_gpu=True)

INFO:pytorch_lightning.utilities.seed:Global seed set to 42
INFO:pytorch_lightning.utilities.distributed:GPU available: True, used: True
INFO:pytorch_lightning.utilities.distributed:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.distributed:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.accelerators.gpu:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 222 M 
-----------------------------------------------------
222 M     Trainable params
0         Non-trainable params
222 M     Total params
891.614   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.seed:Global seed set to 42


Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

In [30]:
import os

# Directory path
directory = '/content/outputs'

# List files in the directory
files = os.listdir(directory)
print(files)


['simplet5-epoch-2-train-loss-4.2653-val-loss-4.5896', 'simplet5-epoch-1-train-loss-4.6048-val-loss-4.7497', 'simplet5-epoch-2-train-loss-4.3053-val-loss-4.5494', 'simplet5-epoch-0-train-loss-5.1713-val-loss-4.9548', 'simplet5-epoch-1-train-loss-4.6371-val-loss-4.7227', 'simplet5-epoch-0-train-loss-5.2133-val-loss-4.887']


In [31]:
model.load_model("t5","outputs/simplet5-epoch-2-train-loss-4.3053-val-loss-4.5494", use_gpu=False)

In [32]:
data = pd.read_csv("/content/drive/MyDrive/fns_2020.csv")

In [33]:
test = preprocess_text(data.iloc[600][1])

In [47]:
test[:100]

'juli pm proof pet home group plc annual report account year end march london stock exchang pet engag'

In [35]:
model.predict(test[:512])

['year end march london stock exchang pet home group plc annual report account year end march london stock exchang pet engag custom engag colleagu deliv growth juli pm proof pet home group plc annual report account year end march']

# Testing on t5-small (No fine tuning)

In [36]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

# Initialize the T5 model and tokenizer
model = T5ForConditionalGeneration.from_pretrained('t5-small')
tokenizer = T5Tokenizer.from_pretrained('t5-small')

# Your long input text
input_text = data.iloc[600][1]

# Maximum sequence length of the model
max_sequence_length = 512

# Split the text into segments
text_segments = [input_text[i:i + max_sequence_length] for i in range(0, len(input_text), max_sequence_length)]

# Generate summaries for each segment
generated_summaries = []
for segment in text_segments:
    input_ids = tokenizer.encode("summarize: " + segment, return_tensors="pt", max_length=max_sequence_length, truncation=True)
    outputs = model.generate(input_ids)
    summary = tokenizer.decode(outputs[0], skip_special_tokens=True)
    generated_summaries.append(summary)

# Combine the generated summaries
combined_summary = ' '.join(generated_summaries)

print("Combined Summary:\n", combined_summary[:200])


Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/231M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.27k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

Combined Summary:
 2341004 17 july 2014 532 pm proof 9 pets at home is the investorspetsathomecom has been the ultimate pet shop experience for pet lovers and their pets lear leader in the highly attractive £54 billion pet market 1 unique ‘one stop shop’ july 2014 532 pm proof 9 in this report 2014 financial headlines revenues £66 cials chief financial officer's review 42 risks and uncertainties 48 governance board of directors 54 f changes in equity 110 consolidated statement of cash flows 112 company balance sheet 113 da is calculated as group underlying operating profit under ifrs. includes amor fcf adjusted for changes in working capital acquisitions of property plant and equipment and other pets at home website relaunched in january 2014. new portfolio strategic report details our performance and how it has been achieved. the strategic report details our performance investorspetsathomecomourcompanyourservices has established local relationships with schools colleges and pet care m

In [46]:
combined_summary[:100]

'2341004 17 july 2014 532 pm proof 9 pets at home is the investorspetsathomecom has been the ultimate'

In [45]:
data.iloc[600][1][:100]

' 2341004    17 july 2014 532 pm    proof 9\npets at home group plc \nannual report and accounts for th'

In [40]:
print(combined_summary)

2341004 17 july 2014 532 pm proof 9 pets at home is the investorspetsathomecom has been the ultimate pet shop experience for pet lovers and their pets lear leader in the highly attractive £54 billion pet market 1 unique ‘one stop shop’ july 2014 532 pm proof 9 in this report 2014 financial headlines revenues £66 cials chief financial officer's review 42 risks and uncertainties 48 governance board of directors 54 f changes in equity 110 consolidated statement of cash flows 112 company balance sheet 113 da is calculated as group underlying operating profit under ifrs. includes amor fcf adjusted for changes in working capital acquisitions of property plant and equipment and other pets at home website relaunched in january 2014. new portfolio strategic report details our performance and how it has been achieved. the strategic report details our performance investorspetsathomecomourcompanyourservices has established local relationships with schools colleges and pet care market leader with 3

In [42]:
len(data.iloc[600][1])

439305

In [43]:
len(combined_summary)

76576

# Fine tuning for better results

In [ ]:
from transformers import Trainer, TrainingArguments

# Define your training arguments
training_args = TrainingArguments(
    output_dir='./results',  # Directory to save the fine-tuned model
    num_train_epochs=3,
    per_device_train_batch_size=4,
    save_steps=500,
    logging_steps=100
)

# Instantiate the Trainer class
trainer = Trainer(
    model=model,                         # The pre-trained T5 model
    args=training_args,                  # Training arguments
    train_dataset=your_training_dataset  # Provide your training dataset
)

# Start fine-tuning
trainer.train()
